In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

import sys
sys.path.append(f'/media/beans/ssd/bespoke')

from models import EffNet
from constants import *
from imports import *
from rw_dataloader import RealWorldDataloader
from blender_dataloader import BlenderDataloader
from train_utils import run_epoch, eval_rw

torch.__version__, torch.cuda.device_count(), torch.cuda.get_device_name(torch.cuda.current_device())

/home/beans/miniconda3/envs/py38/lib/python3.8/site-packages/albumentations/augmentations/transforms.py:689: FutureWarning: This class has been deprecated. Please use CoarseDropout
  warnings.warn(
/home/beans/miniconda3/envs/py38/lib/python3.8/site-packages/albumentations/augmentations/transforms.py:965: FutureWarning: This class has been deprecated. Please use ImageCompression
  warnings.warn(


('1.10.0', 2, 'NVIDIA GeForce RTX 3090')

In [2]:
import wandb
run_id = wandb.util.generate_id(); run_id

'r8v4v7t6'

In [3]:
wandb.init(id='r8v4v7t6', project="carla", resume="allow")

wandb: Currently logged in as: rgilman33 (use `wandb login --relogin` to force relogin)
wandb: wandb version 0.12.18 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


In [4]:
m = EffNet(model_arch="efficientnet_b3").to(device) # 13M params, 11.6M without RNN, 
sum([torch.numel(p) for p in m.parameters()]) / 1000

13332.809

In [5]:
m.backbone

EfficientNet(
  (conv_stem): Conv2d(3, 40, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
  (bn1): BatchNorm2d(40, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (act1): SiLU(inplace=True)
  (blocks): Sequential(
    (0): Sequential(
      (0): DepthwiseSeparableConv(
        (conv_dw): Conv2d(40, 40, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=40, bias=False)
        (bn1): BatchNorm2d(40, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (act1): SiLU(inplace=True)
        (se): SqueezeExcite(
          (conv_reduce): Conv2d(40, 10, kernel_size=(1, 1), stride=(1, 1))
          (act1): SiLU(inplace=True)
          (conv_expand): Conv2d(10, 40, kernel_size=(1, 1), stride=(1, 1))
          (gate): Sigmoid()
        )
        (conv_pw): Conv2d(40, 24, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn2): BatchNorm2d(24, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (act2): Identity()
 

In [6]:
m.load_state_dict(torch.load(f"{BESPOKE_ROOT}/models/m.torch"), strict=False)

<All keys matched successfully>

In [7]:
scaler = torch.cuda.amp.GradScaler() 
opt = torch.optim.Adam(m.parameters(), lr=1e-4)

In [8]:
BS = 32
dataloader_trn = BlenderDataloader(path_stem="trn", bs=BS)

In [9]:
%%time
#rw_dataloader = RealWorldDataloader()

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 6.44 µs


In [10]:
#wandb.watch(m, log="all")

In [11]:
model_stem = "6.16"
START_E = 0

In [ ]:
n_epochs = 100
log_wandb = True

for e in range(START_E, n_epochs):
    
    # Train
    print(f"EPOCH {e}\n")
    run_epoch(dataloader_trn, m, opt=opt, scaler=scaler, 
              train=True, backwards=True, log_wandb=log_wandb, wandb=wandb, updates_per_epoch=2650)
    
    print("\n\n Eval...")
    
    #eval_rw(rw_dataloader, m, wandb) # will log directly to wandb
        
    # Saving model checkpoint each epoch
    torch.save(m.state_dict(), f"{BESPOKE_ROOT}/models/m_{model_stem}_e{e}.torch")
    

EPOCH 0

{'trn_control_loss': 0.00225865, 'consistency losses/trn_steer_cost': 0.00333221, 'consistency losses/trn_te_loss': 0.01262121, 'consistency losses/trn_torque_delta_loss': 45.77349854, 'logistical/obs_consumed_per_second': 238.87401575, 'logistical/obs_generated_per_second': 112.70866142, 'logistical/data_consumption_ratio': 2.12236915, 'logistical/manual_train_pause': 0.0, 'consistency losses/trn_torque_loss': 97.19614362, 'logistical/max_param': 16.24622536, 'logistical/lr': 0.0001}
{'logistical/obs_consumed_per_second': 241.4375, 'logistical/obs_generated_per_second': 108.359375, 'logistical/data_consumption_ratio': 2.23148062, 'logistical/manual_train_pause': 0.0, 'trn_control_loss': 0.00214221, 'consistency losses/trn_steer_cost': 0.0035114, 'consistency losses/trn_te_loss': 0.01344029, 'consistency losses/trn_torque_loss': 110.24505208, 'consistency losses/trn_torque_delta_loss': 47.31091309, 'logistical/max_param': 16.24389839, 'logistical/lr': 0.0001}
{'logistical/obs_

{'logistical/obs_consumed_per_second': 245.2578125, 'logistical/obs_generated_per_second': 113.71875, 'logistical/data_consumption_ratio': 2.16151134, 'logistical/manual_train_pause': 0.0, 'trn_control_loss': 0.0023821, 'consistency losses/trn_steer_cost': 0.00297333, 'consistency losses/trn_te_loss': 0.01089529, 'consistency losses/trn_torque_loss': 131.27276786, 'consistency losses/trn_torque_delta_loss': 48.62084961, 'logistical/max_param': 16.24225044, 'logistical/lr': 0.0001}
{'logistical/obs_consumed_per_second': 238.203125, 'logistical/obs_generated_per_second': 111.9375, 'logistical/data_consumption_ratio': 2.13160423, 'logistical/manual_train_pause': 0.0, 'trn_control_loss': 0.00206409, 'consistency losses/trn_steer_cost': 0.00240451, 'consistency losses/trn_te_loss': 0.00879127, 'consistency losses/trn_torque_delta_loss': 39.37811279, 'consistency losses/trn_torque_loss': 127.55322266, 'logistical/max_param': 16.24363518, 'logistical/lr': 0.0001}
{'logistical/obs_consumed_per

{'logistical/obs_consumed_per_second': 237.296875, 'logistical/obs_generated_per_second': 114.7421875, 'logistical/data_consumption_ratio': 2.07298277, 'logistical/manual_train_pause': 0.0, 'trn_control_loss': 0.00187308, 'consistency losses/trn_steer_cost': 0.00227911, 'consistency losses/trn_te_loss': 0.00864248, 'consistency losses/trn_torque_loss': 106.01331019, 'consistency losses/trn_torque_delta_loss': 38.7322998, 'logistical/max_param': 16.24563217, 'logistical/lr': 0.0001}
{'logistical/obs_consumed_per_second': 232.6640625, 'logistical/obs_generated_per_second': 110.90625, 'logistical/data_consumption_ratio': 2.10031131, 'logistical/manual_train_pause': 0.0, 'trn_control_loss': 0.0018818, 'consistency losses/trn_steer_cost': 0.00224674, 'consistency losses/trn_te_loss': 0.00834601, 'consistency losses/trn_torque_loss': 70.1265625, 'consistency losses/trn_torque_delta_loss': 39.87438965, 'logistical/max_param': 16.24526215, 'logistical/lr': 0.0001}
{'logistical/obs_consumed_per

{'logistical/obs_consumed_per_second': 232.703125, 'logistical/obs_generated_per_second': 104.6953125, 'logistical/data_consumption_ratio': 2.23049736, 'logistical/manual_train_pause': 0.0, 'trn_control_loss': 0.0017695, 'consistency losses/trn_steer_cost': 0.00207713, 'consistency losses/trn_te_loss': 0.00792957, 'consistency losses/trn_torque_delta_loss': 36.29449463, 'consistency losses/trn_torque_loss': 91.78872283, 'logistical/max_param': 16.24393272, 'logistical/lr': 0.0001}
{'logistical/obs_consumed_per_second': 239.3203125, 'logistical/obs_generated_per_second': 97.421875, 'logistical/data_consumption_ratio': 2.46008669, 'logistical/manual_train_pause': 0.000625, 'trn_control_loss': 0.00190377, 'consistency losses/trn_steer_cost': 0.00204616, 'consistency losses/trn_te_loss': 0.00774389, 'consistency losses/trn_torque_delta_loss': 37.49975586, 'consistency losses/trn_torque_loss': 36.63235294, 'logistical/max_param': 16.24411392, 'logistical/lr': 0.0001}
{'logistical/obs_consum

{'logistical/obs_consumed_per_second': 235.203125, 'logistical/obs_generated_per_second': 102.0234375, 'logistical/data_consumption_ratio': 2.31039843, 'logistical/manual_train_pause': 7.812e-05, 'trn_control_loss': 0.00178391, 'consistency losses/trn_steer_cost': 0.00203661, 'consistency losses/trn_te_loss': 0.00753355, 'consistency losses/trn_torque_delta_loss': 40.31451416, 'consistency losses/trn_torque_loss': 101.48535156, 'logistical/max_param': 16.24110985, 'logistical/lr': 0.0001}
{'logistical/obs_consumed_per_second': 239.6328125, 'logistical/obs_generated_per_second': 106.8203125, 'logistical/data_consumption_ratio': 2.25492531, 'logistical/manual_train_pause': 0.0, 'trn_control_loss': 0.00187992, 'consistency losses/trn_steer_cost': 0.00226257, 'consistency losses/trn_te_loss': 0.00837558, 'consistency losses/trn_torque_delta_loss': 40.00024414, 'consistency losses/trn_torque_loss': 113.04361979, 'logistical/max_param': 16.2396965, 'logistical/lr': 0.0001}
{'logistical/obs_c

...

{'logistical/obs_consumed_per_second': 237.109375, 'logistical/obs_generated_per_second': 105.515625, 'logistical/data_consumption_ratio': 2.247072, 'logistical/manual_train_pause': 0.0, 'trn_control_loss': 0.00197674, 'consistency losses/trn_steer_cost': 0.00209695, 'consistency losses/trn_te_loss': 0.0079383, 'consistency losses/trn_torque_delta_loss': 37.65777588, 'consistency losses/trn_torque_loss': 43.82161458, 'logistical/max_param': 16.23589134, 'logistical/lr': 0.0001}
{'logistical/obs_consumed_per_second': 234.0859375, 'logistical/obs_generated_per_second': 100.4921875, 'logistical/data_consumption_ratio': 2.32741224, 'logistical/manual_train_pause': 0.0, 'trn_control_loss': 0.00206523, 'consistency losses/trn_steer_cost': 0.00239354, 'consistency losses/trn_te_loss': 0.0090458, 'consistency losses/trn_torque_loss': 118.58796296, 'consistency losses/trn_torque_delta_loss': 40.03125, 'logistical/max_param': 16.23853874, 'logistical/lr': 0.0001}
{'logistical/obs_consumed_per_se

{'logistical/obs_consumed_per_second': 239.140625, 'logistical/obs_generated_per_second': 101.171875, 'logistical/data_consumption_ratio': 2.378196, 'logistical/manual_train_pause': 0.000625, 'trn_control_loss': 0.0018855, 'consistency losses/trn_steer_cost': 0.00206687, 'consistency losses/trn_te_loss': 0.00790152, 'consistency losses/trn_torque_loss': 59.421875, 'consistency losses/trn_torque_delta_loss': 37.07440186, 'logistical/max_param': 16.23782921, 'logistical/lr': 0.0001}
{'logistical/obs_consumed_per_second': 242.75, 'logistical/obs_generated_per_second': 107.3515625, 'logistical/data_consumption_ratio': 2.26611425, 'logistical/manual_train_pause': 0.0, 'trn_control_loss': 0.00185003, 'consistency losses/trn_steer_cost': 0.00220155, 'consistency losses/trn_te_loss': 0.00826437, 'consistency losses/trn_torque_delta_loss': 39.10046387, 'consistency losses/trn_torque_loss': 74.02734375, 'logistical/max_param': 16.23796463, 'logistical/lr': 0.0001}
{'logistical/obs_consumed_per_s

{'logistical/obs_consumed_per_second': 241.5, 'logistical/obs_generated_per_second': 102.015625, 'logistical/data_consumption_ratio': 2.37066005, 'logistical/manual_train_pause': 0.0, 'trn_control_loss': 0.00176777, 'consistency losses/trn_steer_cost': 0.0022243, 'consistency losses/trn_te_loss': 0.00849916, 'consistency losses/trn_torque_delta_loss': 36.81604004, 'consistency losses/trn_torque_loss': 67.89583333, 'logistical/max_param': 16.23931503, 'logistical/lr': 0.0001}
{'logistical/obs_consumed_per_second': 242.1796875, 'logistical/obs_generated_per_second': 102.953125, 'logistical/data_consumption_ratio': 2.35252626, 'logistical/manual_train_pause': 0.0, 'trn_control_loss': 0.00182571, 'consistency losses/trn_steer_cost': 0.00199997, 'consistency losses/trn_te_loss': 0.00760373, 'consistency losses/trn_torque_delta_loss': 37.34088135, 'consistency losses/trn_torque_loss': 92.79464286, 'logistical/max_param': 16.23735809, 'logistical/lr': 0.0001}
{'logistical/obs_consumed_per_sec

{'logistical/obs_consumed_per_second': 236.6640625, 'logistical/obs_generated_per_second': 106.9453125, 'logistical/data_consumption_ratio': 2.21674013, 'logistical/manual_train_pause': 0.0, 'trn_control_loss': 0.00197972, 'consistency losses/trn_steer_cost': 0.00249865, 'consistency losses/trn_te_loss': 0.00950208, 'consistency losses/trn_torque_delta_loss': 40.85345459, 'consistency losses/trn_torque_loss': 101.52455357, 'logistical/max_param': 16.23621941, 'logistical/lr': 0.0001}
{'logistical/obs_consumed_per_second': 236.9375, 'logistical/obs_generated_per_second': 108.078125, 'logistical/data_consumption_ratio': 2.19504288, 'logistical/manual_train_pause': 0.0, 'trn_control_loss': 0.00171126, 'consistency losses/trn_steer_cost': 0.00192307, 'consistency losses/trn_te_loss': 0.00721686, 'consistency losses/trn_torque_delta_loss': 37.90808105, 'consistency losses/trn_torque_loss': 75.18923611, 'logistical/max_param': 16.23799896, 'logistical/lr': 0.0001}
{'logistical/obs_consumed_p

{'logistical/obs_consumed_per_second': 238.50549451, 'logistical/obs_generated_per_second': 106.71428571, 'logistical/data_consumption_ratio': 2.24141032, 'logistical/manual_train_pause': 0.0, 'trn_control_loss': 0.00188694, 'consistency losses/trn_steer_cost': 0.00204106, 'consistency losses/trn_te_loss': 0.00746626, 'consistency losses/trn_torque_delta_loss': 38.87100694, 'consistency losses/trn_torque_loss': 48.24759615}


 Eval...
EPOCH 11

{'trn_control_loss': 0.00172216, 'consistency losses/trn_steer_cost': 0.00198708, 'consistency losses/trn_te_loss': 0.00746158, 'consistency losses/trn_torque_loss': 76.1725, 'consistency losses/trn_torque_delta_loss': 38.86853027, 'logistical/obs_consumed_per_second': 237.30708661, 'logistical/obs_generated_per_second': 112.00787402, 'logistical/data_consumption_ratio': 2.12101968, 'logistical/manual_train_pause': 0.0, 'logistical/max_param': 16.23794174, 'logistical/lr': 0.0001}
{'logistical/obs_consumed_per_second': 240.578125, 'logistical/ob

In [10]:
%%time

eval_rw(rw_dataloader, m, wandb) # will log directly to wandb

/home/beans/miniconda3/envs/py38/lib/python3.8/site-packages/torch/utils/checkpoint.py:25: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn("None of the inputs have requires_grad=True. Gradients will be None")


CPU times: user 14.3 s, sys: 417 ms, total: 14.7 s
Wall time: 14 s


In [13]:
torch.save(m.state_dict(), f"{BESPOKE_ROOT}/models/m_{model_stem}_e16.torch")